In [1]:
import os, random, matplotlib, pickle, yaml, hdbscan
import numpy as np
from sklearn.utils import shuffle
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from glob2 import glob

In [2]:
skeleton = [
    [0,1,2,3],
    [4,5,6,7],
    [8,9,10],
    [11,12,13],
    [14,15,16],
    [17,18,19,20],
    [21,22,23],
    [24,25,26],
    [27,28,29]
]
skeleton_color=['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive', 'tab:cyan']

In [3]:
result_path = "../results/round3"
with open(f"{result_path}/INFO.yaml") as f:
    INFO = yaml.load(f, Loader=yaml.FullLoader)
    INFO_values = list(INFO.values())
    INFO_values.sort(key=lambda x: x['order'])

In [ ]:
config_path = ".."
with open(f"{config_path}/config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
root_path = ".."
rot_bp, angles, limbs = [], [], []
angle_power, limb_power = [], []
all_embed = []
all_postural_embed, marker_postural_embed, angle_postural_embed, limb_postural_embed = [], [], [], []
all_kinematic_embed, marker_kinematic_embed, limb_kinematic_embed, angle_kinematic_embed = [], [], [], []

for file in tqdm(INFO_values):
    rot_bp.append( np.load(f"{root_path}/{file['directory']}/rotated_bodypoints.npy") )
    angles.append( np.load(f"{root_path}/{file['directory']}/angles.npy") )
    limbs.append( np.load(f"{root_path}/{file['directory']}/limbs.npy") )
    angle_power.append( np.load(f"{root_path}/{file['directory']}/angle_power.npy") )
    limb_power.append( np.load(f"{root_path}/{file['directory']}/limb_power.npy") )
    
    all_embed.append( np.load(f"{root_path}/{file['directory']}/all_embeddings.npy") )
    
    all_postural_embed.append( np.load(f"{root_path}/{file['directory']}/all_postural_embeddings.npy") )
#     marker_postural_embed.append( np.load(f"{root_path}/{file['directory']}/marker_postural_embeddings.npy") )
#     angle_postural_embed.append( np.load(f"{root_path}/{file['directory']}/angle_postural_embeddings.npy") )
#     limb_postural_embed.append( np.load(f"{root_path}/{file['directory']}/limb_postural_embeddings.npy") )
    
    all_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/all_kinematic_embeddings.npy") )
#     marker_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/marker_kinematic_embeddings.npy") )
#     limb_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/marker_limb_embeddings.npy") )
#     angle_kinematic_embed.append( np.load(f"{root_path}/{file['directory']}/marker_angle_embeddings.npy") )
    
rot_bp = np.concatenate(rot_bp)
angles = np.concatenate(angles)
limbs = np.concatenate(limbs)
angle_power = np.concatenate(angle_power)
limb_power = np.concatenate(limb_power)

all_embed = np.concatenate(all_embed)

all_postural_embed = np.concatenate(all_postural_embed)
# marker_postural_embed = np.concatenate(marker_postural_embed)
# angle_postural_embed = np.concatenate(angle_postural_embed)
# alllimb_postural_embed_embed = np.concatenate(limb_postural_embed)

all_kinematic_embed = np.concatenate(all_kinematic_embed)
# marker_kinematic_embed = np.concatenate(marker_kinematic_embed)
# limb_kinematic_embed = np.concatenate(limb_kinematic_embed)
# angle_kinematic_embed = np.concatenate(angle_kinematic_embed)

<hr>
<hr>

In [ ]:
# Figure Information
max_freq_idx = np.argmax(angle_power[:,:,:], axis=1)
max_freq = freq[max_freq_idx]
num_freq = len(freq)

# Freq Color Scheme
freq_color_palette = sns.color_palette('mako_r', num_freq)
cmap1, norm1 = colors.from_levels_and_colors(freq, freq_color_palette, extend="max")

# Figures
fig, ax = plt.subplots(3,4, figsize=(20,15))
for angle_idx in tqdm(range(12)):
    fig_title = f"{config['angle_labels'][angle_idx]} Frequency"
    freq_cluster_colors = np.array([freq_color_palette[f] for f in max_freq_idx[:,angle_idx]])

    ax[angle_idx%3, angle_idx//3].scatter(all_embed[:,0], all_embed[:,1], color=freq_cluster_colors, alpha=0.005, s=1.5)  
    ax[angle_idx%3, angle_idx//3].set(xlabel='Component 1', ylabel='Component 2', title=fig_title, xlim=region_x_lim, ylim=region_y_lim)

plt.show()

In [ ]:
max_freq, min_freq = config['fps']/2, 1 # Nyquist Frequency
freq = max_freq*2**(-1*np.log2(max_freq/min_freq)*
    (np.arange(config['f_bin'],0,-1)-1)/(config['f_bin']-1))

In [ ]:
# Figure Information
max_freq_idx = np.argmax(angle_power[:,:,:], axis=1)
max_freq = freq[max_freq_idx]
num_freq = len(freq)

# Freq Color Scheme
freq_color_palette = sns.color_palette('mako_r', num_freq)
cmap1, norm1 = colors.from_levels_and_colors(freq, freq_color_palette, extend="max")

# Figures
fig, ax = plt.subplots(3,4, figsize=(28,21))
for angle_idx in tqdm(range(12)):
    fig_title = f"{config['angle_labels'][angle_idx]} Frequency"
    freq_cluster_colors = np.array([freq_color_palette[f] for f in max_freq_idx[:,angle_idx]])

    ax[angle_idx%3, angle_idx//3].scatter(all_embed[:,0], all_embed[:,1], color=freq_cluster_colors, alpha=0.005, s=0.5)  
    ax[angle_idx%3, angle_idx//3].set(xlabel='Component 1', ylabel='Component 2', title=fig_title, xlim=region_x_lim, ylim=region_y_lim)

plt.show()

In [ ]:
# Figure Information
max_freq_idx = np.argmax(limb_power[:,:,:], axis=1)
max_freq = freq[max_freq_idx]
num_freq = len(freq)

# Freq Color Scheme
freq_color_palette = sns.color_palette('mako_r', num_freq)
cmap1, norm1 = colors.from_levels_and_colors(freq, freq_color_palette, extend="max")

# Figures
fig, ax = plt.subplots(3,4, figsize=(28,21))
for angle_idx in tqdm(range(12)):
    fig_title = f"{config['angle_labels'][angle_idx]} Frequency"
    freq_cluster_colors = np.array([freq_color_palette[f] for f in max_freq_idx[:,angle_idx]])

    ax[angle_idx%3, angle_idx//3].scatter(all_embed[:,0], all_embed[:,1], color=freq_cluster_colors, alpha=0.005, s=0.5)  
    ax[angle_idx%3, angle_idx//3].set(xlabel='Component 1', ylabel='Component 2', title=fig_title, xlim=region_x_lim, ylim=region_y_lim)
plt.show()